<div style="text-align: center;">
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# H-Series Resource Visibility

To run quantum programs on H-Series emulators and devices, users must submit jobs using `qnexus`. A nexus exeucte job targetting H-Series consists of a `circuit` and a `result`. Circuits contain the instructions for running a program on the quantum computer, and are submitted to a user-specified emulator or device. Once the circuit has completed execution on H-Series, a result is available to view by the end user. There is no expiration date on the retention period of nexus jobs. In other words, nexus jobs are stored on database forever.

Jobs submitted to the H-Series emulators and devices enter a fair queuing system. Jobs will wait in the queue until they run on the target device. A fair queuing process is used to ensure each organization’s queue is equally represented for machine access. The jobs submitted by users in the same organization are executed in the submission order. If users submit a job to a specific machine that is not available, the jobs will remain at the top of an organization’s queue until that machine is available. Machines do not need to be online when submitting jobs. Users are encouraged to submit jobs at their convenience. The H-Series quantum computers are periodically taken down for upgrades. If a job is submitted while the machine is in an upgrade cycle, it will remain in the queue and run when the machine is back online. Users are able and encouraged to submit jobs to the queue.

The [Quantinuum Nexus user portal]([https://nexus.quantinuuum.com]) displays all jobs submitted to the queue for a specific user. [`qnexus`](https://docs.quantinuum.com/nexus) can be used to programmatically access resource visibility. The *Resource Visibility in the User Portal* section narrates how project-level, job-level and device-level data can be displayed. At the device-level, calendar data is available on when H-Series is operational. `qnexus` is used to show job-level and project-level visibility in addition to quota visibility.

**Contents**
* [Queuing System](#Queuing-System)
* [Minimize H-Series Queue Time](#Minimize-H-Series-Queue-Time)
* [Resource Visibility in the User Portal](#Queue-Visibility-in-the-User-Portal)
* [Resource Visibility with `qnexus`](#Queue-Visibility-with-`qnexus`)

## Queuing System

### Nexus Queuing System

Nexus operates a *First-In First-Out* (FIFO) queue for all compilation jobs, in addition to execution jobs submitted to nexus-hosted emulators and simulators. Compilation jobs have a 5 minute window before termination and job failure. All jobs submitted to nexus for execution on H-Series-hosted targets are submitted to a *first-in, first-out* queue within nexus. All jobs submitted to nexus will be directed to H-Series in order of submission date - jobs submitted earlier will be submitted first to H-Series. User and group priorities only impact job selection within the H-Series Fair Queue.

### H-Series Fair-share Queuing System

The fair-share queue fairly distributes access opportunities to H-Series quantum computers and emulators across users from all organizations. The fair-share queue is an important step in the H-Series software stack for scheduling jobs on H-Series and maximizing compute on the hardware.

### Job Selection

Job selection with the fair-share queue is determined by H-System Quantum Credits (HQCs) accumulation per organization, group priority within an organization, user priority within a group and submission date for all jobs of a selected user.

1. HQCs accumulation
    * An organization from all organizations submitting to a target is selected based on the HQC accumulation, how many HQCs have been consumed by the organization, within the Fair Queue Time Window (FQTW).
    * The FQTW is defined to be 2-4 hours.
1. Group Priority
    * Within the selected organization, jobs from User Groups are ranked and chosen according to group priority. 
    * If multiple groups have the same priority, HQC accumulation is used as a secondary criterion to select jobs. Jobs will run first for groups with lower HQC accumulation.
1. User Priority
    * Once a group is selected, the highest priority user is selected.
1. Submission Date
    * The job with the oldest submission date of the selected user begins execution.
1. Job Execution
    * The selected job is chunked into slices based on the number of shots requested by the user and the complexity of the job. Job complexity is determined by the number of two-qubit gates and measurement gates.
    * A slice of a job is a small number of shots that can be executed between calibration runs.
1. Update organization HQC accumulation within FQTW.

<div style="text-align: center;">
         <img src="figures/FQ_1.png" width="800" />
</div>

For jobs submitted with large shot counts, jobs are chunked into `slices` with a smaller shot counts. The number of shots in a slice is dynamically chosen by the H-Series compiler and will vary with the complexity of the circuit. 

A series of system checks are performed before and after each slice. If an error is detected, any suspect results are rejected and the failed chunk shots are rerun at no additional cost.

Once a job is selected and starts running, it is assigned a `start date`. Once the job finishes running, it is assigned a `result date`. For jobs consisting of multiple slices, the time between the start date and result date will include all of the system checks and calibrations that happened in the middle of that job and possibly slices from other jobs in the queue. Users should *not* expect that `result date` less `start date` be the system run-time of the job as this is more than likely not the case.

Partial results can be retrieved for the slices that have completed running on the H-series device or emulator using the [`get_partial_results` method](https://tket.quantinuum.com/extensions/pytket-quantinuum/api.html#pytket.extensions.quantinuum.QuantinuumBackend.get_partial_result) in `pytket`.

Jobs running on the emulator will have a runtime dependent on the number of qubits. A 17-qubit emulation may require 10 minutes to complete while a 25-qubit emulation may require over 24 hours to complete.

Fairness is dictated by the HQC accumulation for each organization. Organizations with premium subscriptions will benefit from slower HQC accumulation and therefore will receive higher priority in the fair-share queue.

### Organizations

Users from the same organization submitting jobs to H-Series compete for access through their organization's queue. Organizations consist of users and groups. Multiple users can be in a group and organizations can have multiple users. Organizations can choose whether or not to use user group priorities or user priorities, controlled by organization administrators. If a user has no association with an existing group, they are assigned the `DEFAULT` group of the organization.

* Organization priority (default: 5) is defined between 1 (highest) and 10 (lowest). 
* Higher priority jobs run before lower priority jobs. 
* Jobs not associated with any group are assigned to the `DEFAULT` group with the default group priority (5).

Long queue times can arise if there are several jobs submitted from the same organization.

Organization administrators cannot modify how the Fair Queue selects organizations.

<div style="text-align: center;">
         <img src="figures/FQ_2.png" width="800" />
</div>

### Job Chunking

For jobs submitted with large shot counts, jobs are chunked into `slices` with a smaller shot counts. The number of shots in a slice is dynamically chosen by the H-Series compiler and will vary with the complexity of the circuit. 

A series of system checks are performed before and after each slice. If an error is detected, any suspect results are rejected and the failed chunk shots are rerun at no additional cost.

The time to solution, the difference between the `result-date` and the `start-date`, is contaminated by chunks of jobs from other organizations. It does not reflect the true execution time on the H-Series device. Batching ensures chunks of jobs from other users do not run during your jobs. Jobs in a batch will run consecutively after each other, after the batch starts.

<div style="text-align: center;">
         <img src="figures/FQ_3.png" width="800" />
</div>

## Minimize H-Series Queue Time

The fair-share queue distributes access to H-Series devices and emulators across multiple organizations. Internally within a selected organization, users can improve the likelihood of their job being selected for execution via communication with the organization administrator. The organization administrator can control user and group priorities to ensure a specific user's jobs start running first within the organization. Group and user priorities can be set by an organization administrator using the user portal or Quantinuum API. The schedule for operations hours on the H-Series device is displayed on the user portal. Emulator targets are readily available to use 24 hours a day. Below are a list of steps a user should follow:

1. Keep jobs in the organization queue.
2. Utilise batching when submitting a large number of jobs.
3. Use User and Group priorities to increase job priority within the organizational queue.
4. Plan QPU and emulator usage around the calendar month.
5. Plan usage around published calendar.
6. Submit to the correct system for the job.

<div style="text-align: center;">
         <img src="figures/FQ_4.png" width="800" />
</div>

## Resource Visibility in the User Portal

The nexus user portal is available at [https://nexus.quantinuum.com/](https://nexus.quantinuum.com/). To access user-specific queue information, valid credentials are required.

### Project View

A projects tab in the top navigation bar leads to a Project view. All projects a user will have access to are available here. The different types of access are available [here](https://docs.quantinuum.com/nexus/concepts/access_control.html).

<div style="text-align: center;">
    <img src="figures/resource_visibility_projects_umui.png" width="800">
</div>

### Jobs View

The user portal also displays all jobs submitted by the user within a project. These can be filtered by a date range for job submission, a name or status.

The user portal job view tabulates the jobs alongside certain properties. These properties are listed below:

* **Job:** A user friendly name attached to the job. Useful for filtering jobs in the job view of the user portal. This is set by the user at the point of job submission.
* **Type:** A `Compile` or `Execute` job.
* **Status:** If the Job is `Queued`, `Running`, `Completed` or `Failed`.
* **Items** Number of items available to view.
* **Backend:** The `BackendConfig` used to submit jobs to the 
* **Submitted by:** Indentity that submitted the job.
* **Submitted:** Time period from submission date to the current period.

<div style="text-align: center;">
    <img src="figures/jobs_view_user_project.png" width="800">
</div>

Job position in the queue is determined by order of submission. Jobs submitted first will be executed first.

The job view in the user portal is user specific. It will not show the position of the user's jobs in the organization queue, or reveal information on jobs submitted by other users in the same organization. 

The circuit data and result data available in a job can be retrieved using the job name.

### Compile Job View

The compile job view shows the job status and the compilation time. In addition, links to the input circuit and the output compiled circuit (job result) are also displayed.

<div style="text-align: center;">
    <img src="figures/compile_job_view.png" width="800" />
</div>

The link to the compiled circuit (job result) redirects to a compiled circuit page displaying a circuit visual. 

<div style="text-align: center;">
    <img src="figures/compile_job_output.png" width="800" />
</div>

### Execute Job View

The execute job view shows the job status and number of shots. Also displayed are links to the input circuit and the job result.

<div style="text-align: center;">
    <img src="figures/execute_job_view.png" width="800" />
</div>

The job result link redirects to the job result page. Displayed are the following quantities: 

* *number of shots*
* *backend snapshot*
* *number of bits*
* most probable measurement *outcome*.

<div style="text-align: center;">
    <img src="figures/execute_job_output.png" width="800" />
</div>

An exportable table is visualized within the nexus portal below. This table shows: 

* the measurement *outcome* on the left;
* the *count* (counting occurence);
* a *probability* of occurence (count in proportion to population size).

<div style="text-align: center;">
    <img src="figures/execute_job_table_output.png" width="800" />
</div>

### Calendar Visualisation

The device tab in the top navigation bar leads to a page collating all available devices. Clicking the `H1-1` entry will lead to a new page dedicated to `H1-1`, displaying the following information:

* Operational calendar for `H1-1`
* Manufacturing specifications, such as two-qubit gate fidelity.
* Native Gates on `H1-1`
* Special features enabled by `H1-1`, such as Mid-circuit measurements and all-to-all connectivity.
* Current status of the device.

The information above is available for all H-Series devices.

<div style="text-align: center;">
    <img src="figures/device-level view.png" width="1500" />
</div>

## Resource Visibility with `qnexus`

`qnexus` is used to submit TKET circuits for compilation and H-Series execution to Nexus. The overall workflow is to construct a measurement circuit for submission with $n$ shots. Once submitted, a job will appear in nexus user portal with the status `Submitted`. Once the job is received by the H-Series queue, the status will be `Queued`. Once the job runs and has `Completed`, a result will be available to the end user. `qnexus` enables retrieval of the result. Even if the H-series job does not complete, results are partially retrievable with `qnexus`.

### Job Visibility

Job status for jobs on H-Series devices can be queried using `qnexus`. The end user should be aware of the different status messages.

* Queued
* Running
* Completed
* Submitted
* Error
* Cancelled

In the cell below, a new nexus project is retrieved, `Resource-Visibility-Demonstration`.

In [1]:
import qnexus

project = qnexus.projects.get_or_create("Resource-Visibility-Demonstration")
qnexus.context.set_active_project(project)

In [2]:
jobname_suffix = qnexus.projects.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")

Next, a circuit is prepared using two native H-Series gates, `OpType.PhasedX` and `OpType.ZZMax`.

In [3]:
from pytket.circuit import Circuit
from numpy.random import random_sample

n_qubits = 10
circuit = Circuit(n_qubits)
px_angles = random_sample(n_qubits)

for i in range(n_qubits):
    circuit.PhasedX(px_angles[i], px_angles[i], i)

for i, j in zip(range(0, n_qubits - 1), range(1, n_qubits)):
    circuit.ZZMax(i, j)

for i in range(n_qubits):
    circuit.PhasedX(px_angles[i], px_angles[i], i)

circuit.measure_all();

In [4]:
ref = qnexus.circuits.upload(circuit=circuit, name="first-circuit")

Specify the `H2-1E` emulator using `QuantinuumConfig`.

In [5]:
config = qnexus.QuantinuumConfig(device_name="H2-1E")

The circuit is submitted to Nexus for compilation.

In [6]:
ref_compile_job = qnexus.start_compile_job(
    circuits=[ref],
    backend_config=config,
    optimisation_level=2,
    name=f"compile-job-{jobname_suffix}"
)

In [7]:
qnexus.jobs.wait_for(ref_compile_job)
ref_compile_circuit = qnexus.jobs.results(ref_compile_job)[0].get_output()

The circuit is submitted to H-series using the [`start_execute_job`]() call with 10 shots. It is recommended to save the reference to disk.

In [8]:
ref_execute_job = qnexus.start_execute_job(
    circuits=[ref],
    backend_config=config,
    n_shots=[100],
    name=f"execute-job-{jobname_suffix}"
)

The `qnexus` method [`jobs.status`]() enables status retrieval from H-series of the job just submitted.

In [19]:
status = qnexus.jobs.status(ref_execute_job)

The `status` variable is an instance of the `qnexus` object [`qnexus.models.job_status.JobStatus`](). 

The `status` property of `JobStatus` returns the enum `Submitted`.

In [20]:
status.status

<StatusEnum.SUBMITTED: 'Circuit has been submitted.'>

The `message` property of `JobStatus` returns the status message as a python string.

In [21]:
print(status.message)

The job is submitted.


For the full `qnexus` workflow, please see this [link]().

### Quota Visibility

In addition to being authenticated, some operations in Nexus require you to have an appropriate Quota. These operations include:

* using the Nexus Lab
* compiling Circuits on Nexus
* Execution on Nexus-hosted H-Series emulators
* storage of Project data in the Nexus Database

Your simulation, compilation and Nexus Lab quota will renew each month, but your database usage quota will not renew. In addition, if you have access to Quantinuum H-series devices you’ll need to have a HQC quota.

To retrieve and display available quotas:

In [22]:
import qnexus

qnexus.quotas.get_all()

[Quota(name='database_usage', description='Total megabytes used to store scientific data.', usage=6.039527999999998, quota='No quota set for user'),
 Quota(name='jupyterhub', description='Total Jupyterhub notebook server running time, in seconds.', usage=14760.0, quota='No quota set for user'),
 Quota(name='simulation', description='Total CPU running time, in seconds.', usage=115961.977201078, quota='No quota set for user'),
 Quota(name='compilation', description='Total CPU running time, in seconds.', usage=23.182771105, quota='No quota set for user')]

Check remaining quotas:

In [23]:
if qnexus.quotas.check_quota(name="compilation"):
    print("Compilation quota OK!")
else:
    print("No more compilation quota!")

Compilation quota OK!


### Project Visibility

One of the core features of Quantinuum Nexus is a database for usefully structuring your scientific data. Data is contained in Projects. Below, data is retrieved for the `Resource-Visibility-Demonstration` project.

In [24]:
import qnexus

project_ref = qnexus.projects.get(name_like="Resource-Visibility-Demonstration")

The project reference can be passed to the `summarize` method to display a summary of the project.

In [25]:
qnexus.projects.summarize(project_ref)

,project,total_jobs,pending_jobs,cancelled_jobs,errored_jobs,completed_jobs
0,Resource-Visibility-Demonstration,8,1,0,1,6


All the jobs for the project can be retrieved using the `get_all` method. Data is displayed below using `df`.

In [27]:
nexus_iterator = qnexus.jobs.get_all(project=project_ref)
nexus_iterator.df()

,name,description,created,modified,job_type,last_status,project,id
0,compile-job-2024_07_25-17_55_26,,2024-07-25 16:59:28.395905+00:00,2024-07-25 16:59:42.668214+00:00,JobType.COMPILE,StatusEnum.COMPLETED,Resource-Visibility-Demonstration,a2176b4a-6281-46e9-87c2-52dd8a64b6df
1,execute-job-2024_07_25-17_55_26,,2024-07-25 17:01:32.131770+00:00,2024-07-25 17:02:03.653537+00:00,JobType.EXECUTE,StatusEnum.ERROR,Resource-Visibility-Demonstration,c6957370-556a-4cc2-a4c3-434324a8d2cc
2,compile-job-2024_07_25-18_03_43,,2024-07-25 17:03:46.873396+00:00,2024-07-25 17:03:49.840371+00:00,JobType.COMPILE,StatusEnum.COMPLETED,Resource-Visibility-Demonstration,72ee0e70-0d07-49cb-8b47-c423d47ce336
3,execute-job-2024_07_25-18_03_43,,2024-07-25 17:05:26.978754+00:00,2024-07-25 17:07:04.087961+00:00,JobType.EXECUTE,StatusEnum.COMPLETED,Resource-Visibility-Demonstration,1acd1529-c140-4fa0-a2c2-0b32502e8859
4,compile-job-2024_07_25-18_11_39,,2024-07-25 17:11:40.421718+00:00,2024-07-25 17:11:42.876238+00:00,JobType.COMPILE,StatusEnum.COMPLETED,Resource-Visibility-Demonstration,5ca5fbb1-5aef-4d14-b51e-912bba625830
5,execute-job-2024_07_25-18_11_39,,2024-07-25 17:11:51.980082+00:00,2024-07-25 17:13:03.988641+00:00,JobType.EXECUTE,StatusEnum.COMPLETED,Resource-Visibility-Demonstration,3c89b971-2f83-4977-b8f0-35e5765b632b
6,compile-job-2024_07_25-18_15_35,,2024-07-25 17:15:37.213636+00:00,2024-07-25 17:15:39.781830+00:00,JobType.COMPILE,StatusEnum.COMPLETED,Resource-Visibility-Demonstration,bf541bdf-c9d6-4a8b-a330-190034f6c3bb
7,execute-job-2024_07_25-18_15_35,,2024-07-25 17:15:48.772359+00:00,2024-07-25 17:26:04.234200+00:00,JobType.EXECUTE,StatusEnum.COMPLETED,Resource-Visibility-Demonstration,a03b328c-ed40-4122-b60d-a1127aba0355


A specific job reference can be selected from the `NexusIterator` instance and the qnexus.jobs module can be used to access job-level data.

In [29]:
ref_compile_job = nexus_iterator.list()[0]

CompileJobRef(id=UUID('a2176b4a-6281-46e9-87c2-52dd8a64b6df'), annotations=Annotations(name='compile-job-2024_07_25-17_55_26', description='', properties=OrderedDict(), created=datetime.datetime(2024, 7, 25, 16, 59, 28, 395905, tzinfo=TzInfo(UTC)), modified=datetime.datetime(2024, 7, 25, 16, 59, 42, 668214, tzinfo=TzInfo(UTC))), job_type=<JobType.COMPILE: 'compile'>, last_status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, last_message='The job is completed.', project=ProjectRef(id=UUID('f0f866d0-73aa-4ea2-a15d-436f38551dd5'), annotations=Annotations(name='Resource-Visibility-Demonstration', description=None, properties=OrderedDict(), created=datetime.datetime(2024, 7, 25, 16, 53, 50, 728257, tzinfo=TzInfo(UTC)), modified=datetime.datetime(2024, 7, 25, 16, 53, 50, 728257, tzinfo=TzInfo(UTC))), contents_modified=datetime.datetime(2024, 7, 25, 17, 26, 4, 234200, tzinfo=TzInfo(UTC)), type='ProjectRef'), type='CompileJobRef')

The annotations field displays useful metadata for the compile job.

In [31]:
ref_compile_job.annotations

Annotations(name='compile-job-2024_07_25-18_15_35', description='', properties=OrderedDict(), created=datetime.datetime(2024, 7, 25, 17, 15, 37, 213636, tzinfo=TzInfo(UTC)), modified=datetime.datetime(2024, 7, 25, 17, 15, 37, 213636, tzinfo=TzInfo(UTC)))

The compilation passes used in the compile job can also be visualized by retrieving the compile job result reference and calling `get_passes`.

In [35]:
ref_compile_result = qnexus.jobs.results(ref_compile_job)[0]

In [36]:
ref_compile_result.get_passes().df()

,pass name,input,output,id
0,DecomposeBoxes,first-circuit,first-circuit,40f763eb-4e23-427c-9f1c-25c2f0b15d27
1,CustomPass,first-circuit,first-circuit,a0332424-6bba-47e8-97f4-0866d0b6a205
2,FullPeepholeOptimise,first-circuit,first-circuit-QuantinuumBackend-2,13e5eb01-d45a-4148-b9b6-6afe5d94b10b
3,NormaliseTK2,first-circuit-QuantinuumBackend-2,first-circuit-QuantinuumBackend-2,23886c3d-e352-44fa-9473-52338e597bd5
4,DecomposeTK2,first-circuit-QuantinuumBackend-2,first-circuit-QuantinuumBackend-3,d0e0e6db-28f4-47c7-82be-149d169ea79a
5,AutoRebase,first-circuit-QuantinuumBackend-3,first-circuit-QuantinuumBackend-4,13c1e648-d60a-443b-b3ba-7b4eab903c4a
6,RemoveRedundancies,first-circuit-QuantinuumBackend-4,first-circuit-QuantinuumBackend-5,6c60215d-a259-402b-912b-8dfc7634623e
7,AutoSquash,first-circuit-QuantinuumBackend-5,first-circuit-QuantinuumBackend-6,7678a67d-cf68-4ee5-8c15-b62e9375e17e
8,RemoveRedundancies,first-circuit-QuantinuumBackend-6,first-circuit-QuantinuumBackend-final,70f261b4-7f4d-4614-98b7-29e5b8358dab
9,FlattenRelabelRegistersPass,first-circuit-QuantinuumBackend-final,first-circuit-QuantinuumBackend-final,a1f2d545-856b-4255-8acf-ee66c6f9a495


The input circuit reference can be obtained using `get_input`.

In [38]:
from pytket.circuit.display import render_circuit_jupyter

ref_input_circuit = ref_compile_result.get_input()
render_circuit_jupyter(ref_input_circuit.download_circuit())

The input circuit reference can be obtained using `get_output`.

In [39]:
from pytket.circuit.display import render_circuit_jupyter

ref_output_circuit = ref_compile_result.get_output()
render_circuit_jupyter(ref_output_circuit.download_circuit())

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>